In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Setup and Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Input, Multiply, Permute, Reshape, Lambda
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Bidirectional, GRU, Layer
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

✓ All imports successful


In [3]:
class AttentionLayer(Layer):
    """
    Custom Attention Layer for Sequence Models

    This layer learns which timesteps in the sequence are most important
    for the classification task. It computes attention weights and returns
    a weighted sum of the input sequence.
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # input_shape: (batch_size, timesteps, features)
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(1,),
            initializer='zeros',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        # inputs shape: (batch_size, timesteps, features)

        # Compute attention scores: (batch_size, timesteps, 1)
        attention_scores = K.tanh(K.dot(inputs, self.W) + self.b)

        # Apply mask if provided (for padded sequences)
        if mask is not None:
            # Expand mask to match attention_scores shape
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, axis=-1)
            # Set masked positions to very negative value
            attention_scores = attention_scores * mask + (1 - mask) * (-1e10)

        # Compute attention weights: (batch_size, timesteps, 1)
        attention_weights = K.softmax(attention_scores, axis=1)

        # Compute weighted sum: (batch_size, features)
        context_vector = K.sum(inputs * attention_weights, axis=1)

        return context_vector

    def compute_output_shape(self, input_shape):
        # Output shape: (batch_size, features)
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        return super(AttentionLayer, self).get_config()

print("✅ Custom Attention Layer defined")

✅ Custom Attention Layer defined


In [4]:
def load_and_filter_fold(i):
    train_dir = f'/content/drive/MyDrive/split_data/fold{i}/train.csv'
    test_dir = f'/content/drive/MyDrive/split_data/fold{i}/test.csv'
    train_df = pd.read_csv(train_dir)
    test_df = pd.read_csv(test_dir)

    train_labels = list(train_df['room'].unique())
    test_labels = list(test_df['room'].unique())
    common_labels = list(set(train_labels) & set(test_labels))

    train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
    test_df = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

    return train_df, test_df

# Load all 4 folds
train_df_1, test_df_1 = load_and_filter_fold(1)
train_df_2, test_df_2 = load_and_filter_fold(2)
train_df_3, test_df_3 = load_and_filter_fold(3)
train_df_4, test_df_4 = load_and_filter_fold(4)

print("✓ All folds loaded")

✓ All folds loaded


In [5]:
def set_seeds(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

def create_room_groups(df):
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['room_group'] = (df['room'] != df['room'].shift()).cumsum()
    return df

def create_beacon_count_vectors(df):
    """Aggregates readings into 1s vectors. Handles data with or without 'room_group'."""
    vectors = []
    has_groups = 'room_group' in df.columns

    for _, group in df.groupby('timestamp'):
        beacon_counts = group['mac address'].value_counts()
        total_readings = len(group)

        vector = [0.0] * 23
        for beacon_id, count in beacon_counts.items():
            if 1 <= beacon_id <= 23:
                vector[int(beacon_id) - 1] = count / total_readings

        entry = {
            'timestamp': group['timestamp'].iloc[0],
            'room': group['room'].iloc[0],
            'beacon_vector': vector
        }

        if has_groups:
            entry['room_group'] = group['room_group'].iloc[0]

        vectors.append(entry)

    return pd.DataFrame(vectors)

def create_sequences_from_groups(vector_df, min_length=3, max_length=50):
    """Used for Training: Creates clean sequences where the room is constant."""
    sequences = []
    labels = []

    for (room, room_group), group in vector_df.groupby(['room', 'room_group']):
        group = group.sort_values('timestamp').reset_index(drop=False)
        seq_length = len(group)

        if seq_length < min_length:
            continue

        if seq_length > max_length:
            group = group.tail(max_length)

        sequence = [row['beacon_vector'] for _, row in group.iterrows()]
        sequences.append(sequence)
        labels.append(room)

    return sequences, labels

print("✅ Basic functions defined")

✅ Basic functions defined


In [6]:
def build_bidirectional_gru_model_with_deep_attention(input_shape, num_classes):
    """
    Deep Bidirectional GRU Architecture with Attention

    Architecture:
    1. Masking layer
    2. First Bi-GRU (128 units) with return_sequences=True
    3. Dropout (0.3)
    4. Second Bi-GRU (64 units) with return_sequences=True
    5. Dropout (0.3)
    6. Attention Layer - aggregates the deep sequence features
    7. Dense layers + Output

    This version keeps both Bi-GRU layers and adds attention on top.
    Proven to be the most stable architecture.
    """
    inputs = Input(shape=input_shape, name='input_layer')

    # Masking for padded sequences
    masked = Masking(mask_value=0.0, name='masking')(inputs)

    # First Bi-GRU layer
    gru1 = Bidirectional(
        GRU(128, return_sequences=True, name='gru_layer_1'),
        name='bidirectional_gru_1'
    )(masked)
    gru1 = Dropout(0.3, name='dropout_1')(gru1)

    # Second Bi-GRU layer
    gru2 = Bidirectional(
        GRU(64, return_sequences=True, name='gru_layer_2'),
        name='bidirectional_gru_2'
    )(gru1)
    gru2 = Dropout(0.3, name='dropout_2')(gru2)

    # Attention mechanism
    attention_output = AttentionLayer(name='attention_layer')(gru2)

    # Dense layers for classification
    dense1 = Dense(32, activation='relu', name='dense_1')(attention_output)
    dense1 = Dropout(0.2, name='dropout_3')(dense1)

    # Output layer
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(dense1)

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name='Deep_BiGRU_with_Attention')

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

print("✅ Deep Attention model architecture defined")

✅ Deep Attention model architecture defined


In [7]:
def create_extended_multidirectional_windows(vector_df):
    """
    Create 7 types of sliding windows for extended multi-directional prediction

    Directions:
    1. backward_10:  [i-9 to i]     - 10s history, predict at i
    2. centered_10:  [i-4 to i+5]   - 10s centered, predict at i
    3. forward_10:   [i to i+9]     - 10s future, predict at i
    4. backward_15:  [i-14 to i]    - 15s history (more context)
    5. forward_15:   [i to i+14]    - 15s future (earlier transition detection)
    6. asymm_past:   [i-11 to i+3]  - 12s past + 4s future (transition from old room)
    7. asymm_future: [i-3 to i+11]  - 4s past + 12s future (entering new room)

    Returns:
        Dictionary with direction names as keys
        Each contains: (sequences, labels, valid_indices)
    """
    # Ensure chronological order and group by day
    vector_df['dt'] = pd.to_datetime(vector_df['timestamp'])
    vector_df['date'] = vector_df['dt'].dt.date

    results = {
        'backward_10': {'sequences': [], 'labels': [], 'indices': []},
        'centered_10': {'sequences': [], 'labels': [], 'indices': []},
        'forward_10': {'sequences': [], 'labels': [], 'indices': []},
        'backward_15': {'sequences': [], 'labels': [], 'indices': []},
        'forward_15': {'sequences': [], 'labels': [], 'indices': []},
        'asymm_past': {'sequences': [], 'labels': [], 'indices': []},
        'asymm_future': {'sequences': [], 'labels': [], 'indices': []},
    }

    for _, day_group in vector_df.groupby('date'):
        day_group = day_group.sort_values('timestamp').reset_index(drop=True)
        vectors = list(day_group['beacon_vector'])
        rooms = list(day_group['room'])
        n = len(vectors)

        for i in range(n):
            if i >= 9:
                window = vectors[i - 9 : i + 1]
                results['backward_10']['sequences'].append(window)
                results['backward_10']['labels'].append(rooms[i])
                results['backward_10']['indices'].append((day_group['date'].iloc[0], i))

            if i >= 4 and i + 5 < n:
                window = vectors[i - 4 : i + 6]
                results['centered_10']['sequences'].append(window)
                results['centered_10']['labels'].append(rooms[i])
                results['centered_10']['indices'].append((day_group['date'].iloc[0], i))

            if i + 9 < n:
                window = vectors[i : i + 10]
                results['forward_10']['sequences'].append(window)
                results['forward_10']['labels'].append(rooms[i])
                results['forward_10']['indices'].append((day_group['date'].iloc[0], i))

            if i >= 14:
                window = vectors[i - 14 : i + 1]
                results['backward_15']['sequences'].append(window)
                results['backward_15']['labels'].append(rooms[i])
                results['backward_15']['indices'].append((day_group['date'].iloc[0], i))

            if i + 14 < n:
                window = vectors[i : i + 15]
                results['forward_15']['sequences'].append(window)
                results['forward_15']['labels'].append(rooms[i])
                results['forward_15']['indices'].append((day_group['date'].iloc[0], i))

            if i >= 11 and i + 3 < n:
                window = vectors[i - 11 : i + 4]
                results['asymm_past']['sequences'].append(window)
                results['asymm_past']['labels'].append(rooms[i])
                results['asymm_past']['indices'].append((day_group['date'].iloc[0], i))

            if i >= 3 and i + 11 < n:
                window = vectors[i - 3 : i + 12]
                results['asymm_future']['sequences'].append(window)
                results['asymm_future']['labels'].append(rooms[i])
                results['asymm_future']['indices'].append((day_group['date'].iloc[0], i))

    return results

print("✅ Extended multi-directional window function defined (7 directions)")

✅ Extended multi-directional window function defined (7 directions)


In [8]:
def train_ensemble_models_with_seeds(train_df, seed_list, verbose=False):
    """
    Train ONE model for EACH seed in seed_list

    Args:
        train_df: Training dataframe
        seed_list: List of seeds (e.g., [42, 1009, 2503, 4001, 5501, 7507, 9001])
        verbose: Print progress

    Returns:
        models: List of trained models (one per seed)
        label_encoder: Fitted label encoder
    """
    if verbose:
        print(f"  Training {len(seed_list)} models (one per seed)...")

    # Prepare data (same for all models)
    train_df_grouped = create_room_groups(train_df)
    train_vector_df = create_beacon_count_vectors(train_df_grouped)
    X_train_seq, y_train_labels = create_sequences_from_groups(train_vector_df, max_length=50)

    # Encode labels
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train_labels)

    # Pad sequences
    X_train_padded = pad_sequences(X_train_seq, maxlen=50, padding='post', dtype='float32', value=0.0)

    # Compute class weights
    class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

    # Train one model per seed
    models = []
    for seed in seed_list:
        set_seeds(seed)

        if verbose:
            print(f"    Training model with seed {seed:5d}...", end=" ")

        # Build deep attention model
        model = build_bidirectional_gru_model_with_deep_attention(
            input_shape=(50, 23),
            num_classes=len(label_encoder.classes_)
        )

        # Callbacks
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=0, min_lr=1e-6)

        # Train
        model.fit(
            X_train_padded, y_train,
            epochs=30,
            batch_size=32,
            class_weight=class_weights,
            callbacks=[early_stop, reduce_lr],
            verbose=0
        )

        models.append(model)

        if verbose:
            print("✓")

    return models, label_encoder

print("✅ Ensemble training function defined (one model per seed)")

✅ Ensemble training function defined (one model per seed)


In [9]:
def predict_single_direction(models, sequences, max_seq_length=50):
    """
    Get ensemble predictions for a single direction

    Args:
        models: List of models (7 models from 7 seeds)
        sequences: Input sequences to predict

    Returns:
        ensemble_proba: (n_samples, n_classes) averaged probability matrix
    """
    # Pad sequences
    X_padded = pad_sequences(sequences, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

    # Get predictions from all models
    all_predictions = []
    for model in models:
        proba = model.predict(X_padded, verbose=0)
        all_predictions.append(proba)

    # Average probabilities across all 7 models
    ensemble_proba = np.mean(all_predictions, axis=0)

    return ensemble_proba

def combine_directional_predictions(direction_results, method='confidence_weighted'):
    """
    Combine predictions from multiple directions using confidence weighting

    Args:
        direction_results: Dict with keys for all 7 directions
        method: 'confidence_weighted', 'equal', or 'softmax'

    Returns:
        combined_proba: (n_positions, n_classes) final probability matrix
        position_map: mapping from (date, position) to array index
    """
    all_positions = set()
    direction_names = ['backward_10', 'centered_10', 'forward_10',
                      'backward_15', 'forward_15',
                      'asymm_past', 'asymm_future']

    for direction in direction_names:
        all_positions.update(direction_results[direction]['indices'])

    all_positions = sorted(all_positions)
    position_map = {pos: idx for idx, pos in enumerate(all_positions)}

    n_classes = direction_results['backward_10']['proba'].shape[1]
    n_positions = len(all_positions)

    combined_proba = np.zeros((n_positions, n_classes))
    position_counts = np.zeros(n_positions)

    for direction_name in direction_names:
        direction_data = direction_results[direction_name]
        proba = direction_data['proba']
        indices = direction_data['indices']

        confidences = np.max(proba, axis=1)

        for i, pos in enumerate(indices):
            pos_idx = position_map[pos]

            if method == 'confidence_weighted':
                weight = confidences[i]
                combined_proba[pos_idx] += proba[i] * weight
            elif method == 'equal':
                combined_proba[pos_idx] += proba[i]

            position_counts[pos_idx] += 1 if method == 'equal' else confidences[i]

    for i in range(n_positions):
        if position_counts[i] > 0:
            combined_proba[i] /= position_counts[i]

    return combined_proba, position_map

print("✅ Multi-directional prediction functions defined")

✅ Multi-directional prediction functions defined


In [10]:
def apply_confidence_weighted_voting(predictions_proba, vote_window=5):
    """
    Confidence-weighted temporal voting

    Args:
        predictions_proba: (n_samples, n_classes) probability matrix
        vote_window: window size for voting

    Returns:
        voted_predictions: (n_samples,) final class predictions
    """
    n_samples, n_classes = predictions_proba.shape
    voted_predictions = np.zeros(n_samples, dtype=int)

    for i in range(n_samples):
        half_window = vote_window // 2
        start = max(0, i - half_window)
        end = min(n_samples, i + half_window + 1)

        window_proba = predictions_proba[start:end]
        window_confidences = np.max(window_proba, axis=1)

        weighted_votes = np.zeros(n_classes)
        for j in range(len(window_proba)):
            weighted_votes += window_proba[j] * window_confidences[j]

        voted_predictions[i] = np.argmax(weighted_votes)

    return voted_predictions

print("✅ Temporal voting function defined")

✅ Temporal voting function defined


In [11]:
def run_final_pipeline(train_df, test_df, seed_list,
                       vote_window=5,
                       combination_method='confidence_weighted',
                       verbose=False):
    """
    FINAL PIPELINE: 7 seeds → 7 models → Ensemble

    Pipeline:
    1. Train 7 models (one per seed) with DEEP ATTENTION
    2. Create 7 directional windows
    3. Get predictions for each direction (all 7 models ensemble)
    4. Combine directions using confidence weighting
    5. Apply temporal voting

    Args:
        seed_list: List of 7 seeds (e.g., [42, 1009, 2503, 4001, 5501, 7507, 9001])
        vote_window: Temporal voting window size
        combination_method: 'confidence_weighted' or 'equal'
    """
    # Clear session
    tf.keras.backend.clear_session()

    if verbose:
        print(f"\n  Training {len(seed_list)} models with optimized seeds...")

    # 1. Train 7 Models (one per seed)
    models, label_encoder = train_ensemble_models_with_seeds(
        train_df,
        seed_list=seed_list,
        verbose=verbose
    )

    if verbose:
        print("  Creating extended multi-directional windows (7 directions)...")

    # 2. Prepare Test Data with Multi-Directional Windows
    test_vectors = create_beacon_count_vectors(test_df)
    direction_windows = create_extended_multidirectional_windows(test_vectors)

    if verbose:
        print("  Getting directional predictions (7 models ensemble per direction)...")

    # 3. Get Predictions for Each Direction (all 7 models vote)
    direction_results = {}
    direction_names = ['backward_10', 'centered_10', 'forward_10',
                      'backward_15', 'forward_15',
                      'asymm_past', 'asymm_future']

    for direction_name in direction_names:
        if verbose:
            print(f"    Predicting {direction_name}...", end=" ")

        sequences = direction_windows[direction_name]['sequences']
        # All 7 models predict and ensemble here
        proba = predict_single_direction(models, sequences, max_seq_length=50)

        direction_results[direction_name] = {
            'proba': proba,
            'indices': direction_windows[direction_name]['indices'],
            'labels': direction_windows[direction_name]['labels']
        }

        if verbose:
            avg_conf = np.mean(np.max(proba, axis=1))
            print(f"avg confidence: {avg_conf:.3f}")

    if verbose:
        print(f"  Combining 7 directions using {combination_method}...")

    # 4. Combine Directional Predictions
    combined_proba, position_map = combine_directional_predictions(
        direction_results,
        method=combination_method
    )

    # Get ground truth labels
    y_test = []
    for pos in sorted(position_map.keys()):
        for direction_name in direction_names:
            if pos in direction_results[direction_name]['indices']:
                idx = direction_results[direction_name]['indices'].index(pos)
                y_test.append(direction_results[direction_name]['labels'][idx])
                break

    if verbose:
        print(f"  Applying temporal voting (window={vote_window})...")

    # 5. Apply Confidence-Weighted Temporal Voting
    y_pred_voted_encoded = apply_confidence_weighted_voting(combined_proba, vote_window=vote_window)
    y_pred = label_encoder.inverse_transform(y_pred_voted_encoded)

    # 6. Final Evaluation
    macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    per_class_f1 = f1_score(y_test, y_pred, average=None, labels=label_encoder.classes_, zero_division=0)

    if verbose:
        print(f"  ✓ Macro F1: {macro_f1:.4f}")

    return {
        'macro_f1': macro_f1,
        'per_class_f1': {label: f1 for label, f1 in zip(label_encoder.classes_, per_class_f1)},
        'combination_method': combination_method,
        'n_models': len(seed_list)
    }

print("✅ Complete final pipeline defined (7 seeds → 7 models → ensemble)")

✅ Complete final pipeline defined (7 seeds → 7 models → ensemble)


In [12]:
# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
# 🏆 OPTIMIZED PRIME SEEDS (Winner from seed optimization)
OPTIMIZED_SEEDS = [42, 1009, 2503, 4001, 5501, 7507, 9001]

print("="*80)
print("FINAL EXPERIMENT: 7-MODEL ENSEMBLE WITH OPTIMIZED SEEDS")
print("="*80)
print(f"\nSeeds: {OPTIMIZED_SEEDS}")
print(f"Strategy: Each seed → 1 model → All 7 ensemble together")
print(f"Architecture: Deep Bidirectional GRU with Attention")
print(f"Directions: 7 (backward_10, centered_10, forward_10, backward_15, forward_15, asymm_past, asymm_future)")
print(f"Temporal voting: 5-second window")
print("="*80)

folds = {
    1: (train_df_1, test_df_1),
    2: (train_df_2, test_df_2),
    3: (train_df_3, test_df_3),
    4: (train_df_4, test_df_4)
}

all_fold_results = {}

for fold_num, (train_df, test_df) in folds.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING FOLD {fold_num}")
    print(f"{'='*80}\n")

    result = run_final_pipeline(
        train_df, test_df,
        seed_list=OPTIMIZED_SEEDS,
        vote_window=5,
        combination_method='confidence_weighted',
        verbose=True
    )

    all_fold_results[fold_num] = result

    print(f"\n  Fold {fold_num} Result: Macro F1 = {result['macro_f1']:.4f}")

print("\n" + "="*80)
print("ALL FOLDS COMPLETED!")
print("="*80)

FINAL EXPERIMENT: 7-MODEL ENSEMBLE WITH OPTIMIZED SEEDS

Seeds: [42, 1009, 2503, 4001, 5501, 7507, 9001]
Strategy: Each seed → 1 model → All 7 ensemble together
Architecture: Deep Bidirectional GRU with Attention
Directions: 7 (backward_10, centered_10, forward_10, backward_15, forward_15, asymm_past, asymm_future)
Temporal voting: 5-second window

PROCESSING FOLD 1


  Training 7 models with optimized seeds...
  Training 7 models (one per seed)...
    Training model with seed    42... ✓
    Training model with seed  1009... ✓
    Training model with seed  2503... ✓
    Training model with seed  4001... ✓
    Training model with seed  5501... ✓
    Training model with seed  7507... ✓
    Training model with seed  9001... ✓
  Creating extended multi-directional windows (7 directions)...
  Getting directional predictions (7 models ensemble per direction)...
    Predicting backward_10... avg confidence: 0.609
    Predicting centered_10... avg confidence: 0.609
    Predicting forward_10...

In [14]:
# 📊 FINAL RESULTS SUMMARY

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80 + "\n")

fold_scores = [all_fold_results[i]['macro_f1'] for i in [1, 2, 3, 4]]

print("PER-FOLD RESULTS:")
print("-"*80)
for fold_num in [1, 2, 3, 4]:
    score = all_fold_results[fold_num]['macro_f1']
    print(f"Fold {fold_num}: {score:.4f}")

overall_mean = np.mean(fold_scores)
overall_std = np.std(fold_scores)

print(f"\n{'='*80}")
print(f"OVERALL PERFORMANCE")
print(f"{'='*80}")
print(f"Mean Macro F1: {overall_mean:.4f} ± {overall_std:.4f}")
print(f"Min: {np.min(fold_scores):.4f}")
print(f"Max: {np.max(fold_scores):.4f}")

# Comparison to baselines
print(f"\n{'='*80}")
print("COMPARISON TO PREVIOUS APPROACHES")
print(f"{'='*80}")

baseline_exp2 = 0.4384
baseline_deep3seeds = 0.4438

print(f"\nExperiment 2 (7 directions, no attention): 0.4384")
print(f"Deep Attention (3 seeds × 5 models):      0.4438")
print(f"Seed Optimization (7 seeds × 1 model):    0.4107 (unstable)")
print(f"\n🏆 FINAL (7 seeds → 7 models ensemble):    {overall_mean:.4f}")

improvement_from_exp2 = overall_mean - baseline_exp2
improvement_from_deep = overall_mean - baseline_deep3seeds

print(f"\nImprovement from Exp 2: {improvement_from_exp2:+.4f} ({improvement_from_exp2/baseline_exp2*100:+.2f}%)")
print(f"Improvement from Deep (3 seeds): {improvement_from_deep:+.4f} ({improvement_from_deep/baseline_deep3seeds*100:+.2f}%)")

# Target achievement
target = 0.45
gap_to_target = target - overall_mean

print(f"\n{'='*80}")
print("TARGET ACHIEVEMENT")
print(f"{'='*80}")
print(f"Target: {target:.4f}")
print(f"Current: {overall_mean:.4f}")
print(f"Gap: {gap_to_target:.4f}")

if overall_mean >= target:
    print("\n🎯🎯🎯 TARGET ACHIEVED! 🎯🎯🎯")
    print(f"✅ Exceeded target by {overall_mean - target:+.4f}!")
    print("\n🏆 This is your FINAL PRODUCTION MODEL!")
elif gap_to_target <= 0.005:
    print("\n🎯 SO CLOSE!")
    print(f"   Only {gap_to_target:.4f} away from target!")
    print(f"   This is an excellent result - essentially at target considering variance.")
else:
    print(f"\n✅ Strong performance! {gap_to_target:.4f} away from target")
    print(f"   Possible next steps:")
    print(f"   • Try vote_window tuning (3, 7, 9)")
    print(f"   • Consider larger ensemble (each seed → 3-5 models)")

print("\n" + "="*80)


FINAL RESULTS SUMMARY

PER-FOLD RESULTS:
--------------------------------------------------------------------------------
Fold 1: 0.4773
Fold 2: 0.4324
Fold 3: 0.4240
Fold 4: 0.4329

OVERALL PERFORMANCE
Mean Macro F1: 0.4417 ± 0.0209
Min: 0.4240
Max: 0.4773

COMPARISON TO PREVIOUS APPROACHES

Experiment 2 (7 directions, no attention): 0.4384
Deep Attention (3 seeds × 5 models):      0.4438
Seed Optimization (7 seeds × 1 model):    0.4107 (unstable)

🏆 FINAL (7 seeds → 7 models ensemble):    0.4417

Improvement from Exp 2: +0.0033 (+0.74%)
Improvement from Deep (3 seeds): -0.0021 (-0.48%)

TARGET ACHIEVEMENT
Target: 0.4500
Current: 0.4417
Gap: 0.0083

✅ Strong performance! 0.0083 away from target
   Possible next steps:
   • Try vote_window tuning (3, 7, 9)
   • Consider larger ensemble (each seed → 3-5 models)



In [15]:
# 💾 SAVE RESULTS TO FILE

with open('final_7seed_ensemble_results.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("FINAL PRODUCTION MODEL RESULTS\n")
    f.write("Deep Bidirectional GRU with Attention + 7-Seed Ensemble\n")
    f.write("="*80 + "\n\n")

    f.write("CONFIGURATION:\n")
    f.write("-"*80 + "\n")
    f.write(f"Seeds: {OPTIMIZED_SEEDS}\n")
    f.write("Ensemble Strategy: 7 seeds → 7 models → All ensemble together\n")
    f.write("Architecture: Deep Bi-GRU (128 → 64) + Attention\n")
    f.write("Directions: 7 (backward_10, centered_10, forward_10, backward_15, forward_15, asymm_past, asymm_future)\n")
    f.write("Temporal Voting: 5-second window\n\n")

    f.write("OVERALL RESULTS:\n")
    f.write("-"*80 + "\n")
    f.write(f"Overall Mean: {overall_mean:.4f} ± {overall_std:.4f}\n")
    f.write(f"Range: {np.min(fold_scores):.4f} to {np.max(fold_scores):.4f}\n\n")

    f.write("PER-FOLD RESULTS:\n")
    f.write("-"*80 + "\n")
    for fold_num in [1, 2, 3, 4]:
        result = all_fold_results[fold_num]
        f.write(f"\nFold {fold_num}: {result['macro_f1']:.4f}\n")
        f.write("  Per-Class F1:\n")
        for class_name, f1 in sorted(result['per_class_f1'].items()):
            f.write(f"    {class_name:20s}: {f1:.4f}\n")

    f.write("\n" + "="*80 + "\n")
    f.write("COMPARISON TO BASELINES:\n")
    f.write("-"*80 + "\n")
    f.write("Experiment 2 (7 directions, no attention): 0.4384\n")
    f.write("Deep Attention (3 seeds × 5 models):      0.4438\n")
    f.write(f"Final (7 seeds → 7 models ensemble):       {overall_mean:.4f}\n\n")
    f.write(f"Improvement from Exp 2: {improvement_from_exp2:+.4f}\n")
    f.write(f"Improvement from Deep (3 seeds): {improvement_from_deep:+.4f}\n")

    f.write("\n" + "="*80 + "\n")
    f.write(f"Gap to target (0.45): {gap_to_target:.4f}\n")
    if overall_mean >= target:
        f.write("\n🎯 TARGET ACHIEVED!\n")
    f.write("="*80 + "\n")

print("✅ Results saved to final_7seed_ensemble_results.txt")

print("\n" + "="*80)
print("EXPERIMENT COMPLETE!")
print("="*80)
print(f"\n🏆 Final Performance: {overall_mean:.4f} ± {overall_std:.4f}")
print(f"📊 7 optimized seeds working together!")
if overall_mean >= 0.45:
    print(f"🎯 TARGET ACHIEVED - This is your production model!")
print("="*80)

✅ Results saved to final_7seed_ensemble_results.txt

EXPERIMENT COMPLETE!

🏆 Final Performance: 0.4417 ± 0.0209
📊 7 optimized seeds working together!
